In [29]:
import os
import openai

# .envファイルからAPIキーを読み込む
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

In [30]:
from langchain.chat_models import ChatOpenAI

In [64]:
from langchain.document_loaders import PyPDFLoader 

# PDFの読み込み
file = '../data/GPT-4_Technical_Report.pdf'
loader = PyPDFLoader(file)

# PDFをページごとに分割
pages = loader.load_and_split()

In [32]:
pages[0]

Document(page_content='GPT-4 Technical Report\nOpenAI\x03\nAbstract\nWe report the development of GPT-4, a large-scale, multimodal model which can\naccept image and text inputs and produce text outputs. While less capable than\nhumans in many real-world scenarios, GPT-4 exhibits human-level performance\non various professional and academic benchmarks, including passing a simulated\nbar exam with a score around the top 10% of test takers. GPT-4 is a Transformer-\nbased model pre-trained to predict the next token in a document. The post-training\nalignment process results in improved performance on measures of factuality and\nadherence to desired behavior. A core component of this project was developing\ninfrastructure and optimization methods that behave predictably across a wide\nrange of scales. This allowed us to accurately predict some aspects of GPT-4’s\nperformance based on models trained with no more than 1/1,000th the compute of\nGPT-4.\n1 Introduction\nThis technical report pre

In [34]:
# ベクターストアの作成(既存のドキュメントを構造化して保存するためのデータベース)
from langchain.vectorstores import DocArrayInMemorySearch 
from langchain.embeddings.openai import OpenAIEmbeddings

# ベクターストアの作成
vectorstore = DocArrayInMemorySearch.from_documents(
    documents=pages,
    embedding=OpenAIEmbeddings(disallowed_special=()),
)

In [65]:
# ベクターストアの作成
db = DocArrayInMemorySearch.from_documents(
    documents=pages,
    embedding=OpenAIEmbeddings(),
)

ValueError: Encountered text corresponding to disallowed special token '<|endofprompt|>'.
If you want this text to be encoded as a special token, pass it to `allowed_special`, e.g. `allowed_special={'<|endofprompt|>', ...}`.
If you want this text to be encoded as normal text, disable the check for this token by passing `disallowed_special=(enc.special_tokens_set - {'<|endofprompt|>'})`.
To disable this check for all special tokens, pass `disallowed_special=()`.


In [45]:
# llmの作成
llm = ChatOpenAI(temperature = 0.0, model="gpt-3.5-turbo")

# Retrieverの作成
retriever = vectorstore.as_retriever()

# Q&Aのためのチェインの作成
from langchain.chains import RetrievalQA

qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [46]:
response = qa_stuff.run('この論文を要約して')



> Entering new  chain...

> Finished chain.


In [47]:
print(response)

GPT-4のInstructGPT論文は、人間のフィードバックを用いて大規模な言語モデルを訓練し、指示に従う能力を向上させることに焦点を当てています。著者は、言語モデルを大きくすること自体が、ユーザーの意図に従う能力を向上させるわけではないと指摘しています。大きなモデルは、真実でない、有害な、または単に役に立たない出力を生成することがあります。

この問題に対処するため、著者らは人間のフィードバックを用いて言語モデルを幅広いタスクに対して微調整します。まず、ラベラーが作成したプロンプトと応答のセットから始め、モデルの望ましい振る舞いを示すラベラーのデモンストレーションのデータセットを収集します。彼らは教師あり学習を用いてGPT-3を微調整し、その後、人間のフィードバックに基づいて強化学習を行いモデルをさらに微調整します。その結果、InstructGPTと呼ばれるモデルは、真実性の向上と有害な出力の削減を示し、一般的なNLPデータセットでの性能の低下は最小限です。

著者は、人間のフィードバックを用いた微調整が言語モデルを人間の意図とより一致させるための有望な手法であると結論付けています。


In [48]:
response2 = qa_stuff.run('GPT-3とGPT-4の違いについて、論文を読んで説明してください。')



> Entering new  chain...

> Finished chain.


In [49]:
print(response2)

GPT-4は、GPT-3と比較していくつかの重要な改善が行われています。まず、GPT-4は画像とテキストの入力を受け付け、テキストの出力を生成することができるマルチモーダルモデルです。これにより、GPT-4はより複雑で微妙なシナリオでの自然言語テキストの理解と生成能力が向上しています。

GPT-4は、人間のテストを含むさまざまなプロフェッショナルおよび学術的なベンチマークで人間レベルのパフォーマンスを示しています。たとえば、シミュレートされた司法試験では、GPT-4はテスト受験者の上位10％に入るスコアを達成しています。これに対して、GPT-3.5は下位10％のスコアを示しています。

さらに、GPT-4は従来のNLPベンチマークでも優れたパフォーマンスを発揮しています。MMLUベンチマークでは、GPT-4は既存のモデルを大幅に上回るだけでなく、他の言語でも強力なパフォーマンスを示しています。翻訳されたバリアントでは、GPT-4は26の言語のうち24言語で英語の最先端を超えています。

ただし、GPT-4にはいくつかの制約もあります。まず、完全に信頼性があるわけではなく、事実を「幻覚」させたり、推論エラーを起こすことがあります。また、GPT-4は限られたコンテキストウィンドウを持ち、経験から学習することはありません。さらに、GPT-4はセキュリティの脆弱性を導入するなど、難しい問題に対しても人間と同様に失敗することがあります。

以上が、GPT-3とGPT-4の主な違いです。詳細な情報は、論文を参照してください。


In [50]:
response3 = qa_stuff.run('事実を幻覚させるとはどういうことですか？論文を読んで教えてください。')



> Entering new  chain...

> Finished chain.


In [52]:
print(response3)

「事実を幻覚させる」とは、GPT-4のような言語モデルが、特定の情報源に関連していない、非現実的または真実ではない内容を生成することを指します。これは、モデルがますます説得力があり信じられるようになるにつれて、ユーザーがそれに過度に依存することを引き起こす可能性があります。この傾向は特に有害であり、モデルがユーザーに真実の情報を提供することで信頼を築いた場合により危険となります。また、これらのモデルが社会に統合され、さまざまなシステムの自動化に役立つようになるにつれて、幻覚の傾向は情報の総合的な品質の低下や、自由に利用できる情報の信頼性の低下につながる要因の一つとなります。

GPT-4の幻覚のポテンシャルを評価するために、閉じたドメインとオープンドメインの両方のコンテキストで測定を行いました。閉じたドメインの幻覚は、自動評価（GPT-4をゼロショットクラシファイアとして使用）と人間の評価を使用して測定しました。オープンドメインの幻覚については、事実ではないとフラグが付けられた実世界のデータを収集し、それをレビューして「事実」となるセットを作成しました。これを使用してモデルの生成物を「事実」のセットと比較し、人間の評価を容易にしました。

GPT-4は、ChatGPTなどの以前のモデルからのデータを活用して、幻覚の傾向を減らすためにトレーニングされました。内部評価では、GPT-4のローンチ版は、オープンドメインの幻覚を避ける点で最新のGPT-3.5モデルよりも19パーセントポイント高く、閉じたドメインの幻覚を避ける点で29パーセントポイント高いスコアを示しました。

ただし、この回答は論文の一部の抜粋です。論文全体にはさらなる詳細や結果が含まれています。


In [62]:
response4 = qa_stuff.run('''
論文を日本語で要約してください。ただし、論文の各セクションごとに内容をまとめてください。
要約は、以下の形式で、全セクションに対して要約を出力してください。
各見出しは、要約に対する適切なタイトルに置き換えてください。

## 見出し
要約1

## 見出し
要約2

...

''')



> Entering new  chain...

> Finished chain.


In [63]:
print(response4)

## 要約1: 技術報告の概要
GPT-4は、画像とテキストの入力を受け取り、テキストの出力を生成する大規模なマルチモーダルモデルです。GPT-4は、次のトークンを予測するために事前学習されたTransformerベースのモデルです。ポストトレーニングのアライメントプロセスにより、事実性や望ましい振る舞いへの遵守度の向上が見られます。このプロジェクトの中心的な要素は、さまざまなスケールで予測可能な振る舞いを示すためのインフラストラクチャと最適化手法の開発でした。

## 要約2: 導入
この技術報告では、画像とテキストの入出力を処理する大規模なマルチモーダルモデルであるGPT-4を紹介しています。このようなモデルの開発は、対話システム、テキスト要約、機械翻訳など、さまざまなアプリケーションで使用される可能性があるため、近年の関心と進歩の対象となっています。GPT-4は、より複雑で微妙なシナリオでの自然言語テキストの理解と生成能力を向上させることを目指しています。GPT-4は、人間向けに設計されたさまざまな試験での能力をテストしました。これらの評価では、GPT-4は非常に優れた成績を収め、多くの場合、人間の受験者の中でトップ10％に入るスコアを獲得しています。

## 要約3: モデルの能力と結果
GPT-4は、従来の大規模言語モデルや最先端のシステム（しばしばベンチマーク固有のトレーニングや手作業エンジニアリングを行っている）を上回る性能を発揮します。英語のMMLUベンチマーク[35,36]では、既存のモデルを大幅に上回るだけでなく、他の言語でも強力なパフォーマンスを示します。翻訳されたバリアントのMMLUでは、GPT-4は26の言語のうち24言語で英語の最先端を超えています。モデルの能力と結果については、後のセクションで詳しく説明しています。

## 要約4: プロジェクトの課題と結論
このプロジェクトの主な課題は、さまざまなスケールで予測可能な振る舞いを示すディープラーニングのインフラストラクチャと最適化手法の開発でした。これにより、同様の方法でトレーニングされた小規模な実行に基づいてGPT-4の予想されるパフォーマンスについて予測することができました。GPT-4は、以前のGPTモデルと同様の制約を持っていますが、その能力にもかかわらず、完全に信頼性があるわけで